# Rule: **add_electricity**

**Outputs**

- resources/networks/`elec.nc`

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
######################################## Run parameters
name = '00_ref'
prefix = ''

## `elec.nc`

Load the network and show its components.

In [ ]:
file = f'elec.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
file = f'regions_onshore.geojson'
path = f'{params['rootpath']}/resources/{prefix}/{name}/'
xp.map_add_region(ax, file, path, params['map_add_region'])

### Add regions_offshore
file = f'regions_offshore.geojson'
path = f'{params['rootpath']}/resources/{prefix}/{name}/'
xp.map_add_region(ax, file, path, params['map_add_region'])

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.carriers`

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.generators`

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have generators for each carrier? How many of them have zero capacity?

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Maps

Plot a map showing a particular feature of a generation carrier at each region.

In [ ]:
#################### Parameters

### Select carrier
carrier = 'solar'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom'
feature = 'p_nom_density'
# feature = 'p_nom_max'
# feature = 'p_nom_max_density'
# feature = 'p_nom_max_ratio'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Derived parameters
### Define region file
if 'off' in carrier:
    file_regions = f'regions_offshore.geojson'
else:
    file_regions = f'regions_onshore.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add regions
xp.map_add_region(ax, file_regions, path_regions, params['map_add_region'])

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generators(carrier, n, feature, file_regions, path_regions, ax, params['map_network_generators'], params_local)

### Variable: `n.lines`

Changes:

- New lines defined in TYNDP may have been incorporated.
- Columns 'project_status' and 'tags' have been included
- Column 'capital_cost' has been filled


Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the relationship between line capital costs and line length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.links`

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

### Variable: `n.shapes`

Place `n.shapes` in a geodataFrame and show its content.

In [ ]:
sh = n.shapes
n
sh.head()

### Variable: `n.transformers`

Place `n.transformers` in a dataFrame and show its content.

In [ ]:
tt= n.transformers

tt.head()

How many transformers are there in the network?

In [ ]:
len(tt)

## `regions_onshore.geojson`

Load the file and show its content.

In [ ]:
file = f'regions_onshore.geojson'
path = f'{params['rootpath']}/resources/{prefix}/{name}/'

gdf_regions_onshore = gpd.read_file(path+file)

gdf_regions_onshore.head()

How many regions are there in the file?

In [ ]:
len(gdf_regions_onshore)

## `regions_offshore.geojson`

Load the file and show its content.

In [ ]:
file = f'regions_offshore.geojson'
path = f'{params['rootpath']}/resources/{prefix}/{name}/'

gdf_regions_offshore = gpd.read_file(path+file)

gdf_regions_offshore.head()

How many regions are there in the file?

In [ ]:
len(gdf_regions_offshore)